# 0.0 Imports

## 0.1 Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import inflection
import re
import s3fs 
import json
from datetime import datetime

from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import DBSCAN

from umap import UMAP

from sqlalchemy import create_engine

import gc

/home/ubuntu/.pyenv/versions/3.8.0/envs/insiders_deploy/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 0.2 Helper Functions



In [ ]:
# Defining some plot settings
def jupyter_settings():
    %matplotlib inline
    #%pylab inline
    #matplotlib notebook
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    # display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()

jupyter_settings()

## 0.4 Columns Description



### Attribute Description
- InvoiceNo Invoice number (A 6-digit integral number uniquely assigned to each transaction)
- StockCode Product (item) code
- Description Product (item) name
- Quantity The quantities of each product (item) per transaction
- InvoiceDate The day when each transaction was generated
- UnitPrice Unit price (Product price per unit)
- CustomerID Customer number (Unique ID assigned to each customer)
- Country Country name (The name of the country where each customer resides)

# 1.0 Data Description


In [ ]:
# df1 = pd.read_csv('../data/raw/ecommerce.csv', encoding='ISO-8859-1', sep = ',')
# df1 = pd.read_csv('/home/humberto/DS/insiders_clustering/data/raw/ecommerce.csv', encoding='ISO-8859-1', sep = ',')

# Reading data from S3 Bucket:
path_s3 = 'https://insiders-raw-data-bucket.s3.sa-east-1.amazonaws.com/raw/'
df1 = pd.read_csv( path_s3 + 'ecommerce.csv', encoding='ISO-8859-1')

In [ ]:
# df1_s3.info()

In [ ]:
if df1.shape == (541909, 9):
    print('Dimensions ok:{}'.format(df1.shape))
else:
    print('Dimensions differ from expected: {}'.format(df1.shape) )

## 1.1 Rename Columns

In [ ]:
cols_old = df1.columns.tolist()

cols_new = [inflection.underscore(col) for col in cols_old]
cols_new

In [ ]:
df1.rename(columns = dict(zip(cols_old, cols_new)), inplace = True )

## 1.2 Data Dimensions

In [ ]:
print( 'Number of rows: {}'.format( df1.shape[0] ) )
print( 'Number of cols: {}'.format( df1.shape[1] ) )

## 1.3 Data Types

In [ ]:
df1.dtypes

## 1.4 NA Check

In [ ]:
df1.isna().sum()/df1.shape[0]*100

In [ ]:
df1.drop(columns=['unnamed: 8'], inplace = True)
df1.columns

## 1.5 NA Fillout

In [ ]:
df1.dropna(subset=['customer_id'], axis = 0, inplace =  True)
df1.isna().sum()

## 1.6 Changing Data Types


In [ ]:
df1.head()

In [ ]:
df1.dtypes

In [ ]:
df1['invoice_date'] = pd.to_datetime( df1['invoice_date'], format = '%d-%b-%y' )

df1['customer_id'] = df1['customer_id'].astype(int)

df1.head()

In [ ]:
df1.dtypes

## 1.7 Descriptive Statistics

### 1.7.1 Numerical Attributes

In [ ]:
num_att = df1.select_dtypes(include= ['int','float'])
d1 = pd.DataFrame( num_att.min() ).T
d2 = pd.DataFrame( num_att.max() ).T
d3 = pd.DataFrame( num_att.max() - num_att.min() ).T
d4 = pd.DataFrame( num_att.mean() ).T
d5 = pd.DataFrame( num_att.median() ).T
d6 = pd.DataFrame( num_att.std() ).T
d7 = pd.DataFrame( num_att.skew() ).T
d8 = pd.DataFrame( num_att.kurtosis() ).T

desc = pd.concat( [d1, d2, d3, d4, d5, d6, d7, d8] ).T
desc.columns = ['min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis' ]
desc.style.format( "{:,.2f}" )

# 2.0 Data Filtering

## 2.1 Data Inspection

In [ ]:
df2 = df1.copy()

del df1
gc.collect()

### Stock Code

In [ ]:
# selecting rows with only non digits
df_stock = df2.loc[df2['stock_code'].apply( lambda x: bool(re.search('^[\D]+$', x)) )]
df_stock.head(10)

In [ ]:
# values to remove
drop_stock = df_stock['stock_code'].unique().tolist()

df_stock['stock_code'].value_counts()

### Country

In [ ]:
#countries to drop
drop_country = ['Unspecified', 'European Community']

## 2.2 Data Filtering

In [ ]:
# dropping stock code
df2 = df2.loc[~ df2['stock_code'].isin(drop_stock)]

# dropping countries
df2 = df2.loc[~ df2['country'].isin(drop_country)]


# 3.0 Feature Engineering

Now lets calculate recency, frequency and monetary using only purchases (not considering returns as a interaction)

In [ ]:
# Reading Data
df3 = df2.copy()

# df3 = pd.read_csv('../data/interim/df2.csv', float_precision='round_trip')
# df3['invoice_date'] = pd.to_datetime( df3['invoice_date'] )

# separating returns from purchases
df_returns = df3.loc[ df3['quantity'] < 0 ]
df_purchases = df3.loc[ df3['quantity'] >= 0 ]

del df2
gc.collect()

### Recency

In [ ]:
# days since last purchase (considering first day as day after last invoice)
df_recency = df_purchases[['customer_id', 'invoice_date']].copy()
df_recency = df_recency.drop_duplicates(subset = 'customer_id', keep = 'last')

# using final date as max date + 1
df_recency['recency'] =  (pd.to_datetime('2017-12-08') - df_recency['invoice_date']).dt.days
df_recency.drop(columns=['invoice_date'], inplace = True)


### Average Days Between Purchases

In [ ]:
df_dates = df_purchases[['customer_id', 'invoice_date']].copy()

# creates a column with the dates shifted by 1 to calculate difference betwenn last purchase and previous purchase
df_dates = df_dates.drop_duplicates()
df_dates['previous_date'] = df_dates.groupby('customer_id')['invoice_date'].shift()

# extracts days betwenn last purchase and previous purchase for each pair of dates
df_dates['days'] = df_dates.apply(lambda x : (x['invoice_date'] - x['previous_date']).days , axis = 1 )

# calculating average of days between purchases
df_dates = df_dates[['customer_id', 'days']].groupby('customer_id').mean().reset_index().rename(columns = {'days': 'avg_days_bw_purchases'})

df_dates.fillna(value = 0, inplace = True)
# df_dates.head()

In [ ]:
df_dates = df_purchases[['customer_id', 'invoice_date']].copy()

# creates a column with the dates shifted by 1 to calculate difference betwenn last purchase and previous purchase
df_dates = df_dates.drop_duplicates()
df_dates['previous_date'] = df_dates.groupby('customer_id')['invoice_date'].shift()

# # extracts days betwenn last purchase and previous purchase for each pair of dates
df_dates['days'] = df_dates.apply(lambda x : (x['invoice_date'] - x['previous_date']).days , axis = 1 )

# # calculating average of days between purchases
df_dates = df_dates[['customer_id', 'days']].groupby('customer_id').mean().reset_index().rename(columns = {'days': 'avg_days_bw_purchases'})

df_dates.fillna(value = 0, inplace = True)
# df_dates.isna().sum()
# df_dates[df_dates.customer_id == 12346] #.mean()
# df_dates.head()

### Frequency

In [ ]:
# Frequency
# nº of purchases a customer did
df_freq = df_purchases[['invoice_no', 'customer_id']].drop_duplicates('invoice_no').groupby(['customer_id']).count().reset_index().rename(columns = {'invoice_no' : 'num_purchases'} ) #'invoice_date',  #['num_of_purchases']

# difference between firts and last transaction
# if nº purchases made = 1: date range =  today - first purchase date
# else last purchase - first purchase
func = lambda x: (x.max() - x.min() if (x.max() - x.min()).days != 0 else pd.to_datetime('2017-12-08') - x.min() ).days

# aggregating things
df_date_range = df_purchases[[ 'customer_id', 'invoice_date' ]].groupby('customer_id').agg( func ).reset_index().rename(columns = {'invoice_date' : 'date_range'})

df_freq = df_freq.merge(df_date_range, on = 'customer_id', how = 'left')
df_freq['frequency'] = df_freq['num_purchases']/df_freq['date_range']

# df_freq.head()

### Revenue

In [ ]:
# revenue
# copying from original dataset so that revenue represents net earnings (purchases - returns)
df_revenue = df3[['unit_price','quantity','customer_id']].copy()
df_revenue['quantity'] = df_revenue['quantity'].astype(float)
df_revenue['customer_spend'] = df_revenue['unit_price']*df_revenue['quantity']

# calculating revenue
df_revenue = df_revenue[['customer_id', 'customer_spend']].groupby('customer_id').sum().reset_index().rename(columns = {'customer_spend' : 'revenue'})

# defining number of purchases
df_revenue = df_revenue.merge( df_freq[['customer_id', 'num_purchases']], on = 'customer_id', how = 'left' )

# calculating avg ticket
df_revenue['avg_ticket'] = df_revenue['revenue']/df_revenue['num_purchases']
df_revenue.drop('num_purchases', axis = 1, inplace = True)

# # deleting rows that we have data about return and not about purchase (purchase outside time frame of dataset)
df_revenue = df_revenue[ df_revenue['revenue'] > 0.1 ].copy()
# df_revenue.head()

### Basket Size

In [ ]:
# selecting cols
df_basket = df3[['customer_id', 'invoice_no', 'quantity']].copy()

# counting items per invoice
df_basket = df_basket.groupby(['customer_id', 'invoice_no']).sum().reset_index()

# calculating quantity of items purchased + avg basket size (men of quantities)
df_basket = df_basket.groupby(['customer_id']).agg( qty_items = ('quantity', 'sum'), avg_basket_size = ('quantity', 'mean') ).reset_index()

# renaming
df_basket = df_basket.rename( columns = {'quantity' : 'avg_basket_size'})

# df_basket.head()

### Average Unique Products Purchased Quantity

In [ ]:
# selecting cols
df_unique = df_purchases[['customer_id', 'invoice_no', 'stock_code']].copy()

# counting number of different products per invoice
df_unique = df_unique.groupby(['customer_id', 'invoice_no']).nunique().reset_index()

# calculating mean of items per invoice
df_unique = df_unique.groupby(['customer_id']).mean().reset_index()

# # renaming
df_unique = df_unique.rename( columns = {'stock_code' : 'avg_unique_prods'})

# df_unique.head()

### Returns

In [ ]:
# creating returns variables

df_returns = df_returns[['quantity', 'invoice_date', 'unit_price', 'customer_id', 'invoice_no']]

df_returns['customer_spend'] = df_returns['unit_price']*df_returns['quantity']

# sum and mean of revenue
df_returns_metrics = df_returns[['customer_id', 'customer_spend', 'quantity']].groupby('customer_id').agg( 
    returns_revenue = ('customer_spend', 'sum'),
    avg_return_revenue = ('customer_spend', 'mean'),
    num_returns = ('customer_spend', 'count'),
    qty_returned = ('quantity', 'sum')  )

df_returns_metrics = pd.DataFrame( df_returns_metrics.to_records() )

# df_returns_metrics.head()

In [ ]:
# merging every feature together

df_processed = df_purchases[['customer_id', 'country']].drop_duplicates('customer_id').copy()
df_processed = pd.merge( df_processed, df_recency, on = 'customer_id', how = 'left' )
df_processed = pd.merge( df_processed, df_dates, on = 'customer_id', how = 'left' )
df_processed = pd.merge( df_processed, df_freq, on = 'customer_id', how = 'left' )
df_processed = pd.merge( df_processed, df_revenue, on = 'customer_id', how = 'inner' )
df_processed = pd.merge( df_processed, df_basket, on = 'customer_id', how = 'left' )
df_processed = pd.merge( df_processed, df_unique, on = 'customer_id', how = 'left' )
df_processed = pd.merge( df_processed, df_returns_metrics, on = 'customer_id', how = 'left' )


# filling na as 0 for customers that never returned an item
df_processed.fillna(value = 0, inplace = True)

# df_processed.dropna( inplace = True)
# df_processed.isna().sum()
# df_processed.head(10)
df_processed.shape

In [ ]:
# df_processed.to_csv('../data/processed/df3_processed.csv', index = False)

# 4.0 Exploratory Data Analysis

In [ ]:
# Reading filtered data
# df4 = pd.read_csv('../data/processed/df3_processed.csv')
df4 = df_processed.copy()

del df3, df_processed, df_returns, df_purchases 
gc.collect()

## 4.1 Univariate Analysis

In [ ]:
### Returns

df4.sort_values(['returns_revenue'], ascending = True).head()

# This is propably an anomaly, error, or some data purposely inserted for unknown reasons
# df3[df3.customer_id == 16446]
# df3[df3.customer_id == 12346]

# These customers indeed appear to have returned high quantities
# df3[df3.customer_id == 15749]
# df3[df3.customer_id == 16029]
# df3[df3.customer_id == 12931]

# droping weird behaviour
cust_drop = [16446, 12346]
df4 = df4[ ~ df4['customer_id'].isin(cust_drop)].copy()

In [ ]:
df4.shape

# 5.0 Data Preparation

## 5.1 Numerical Variables Preparation

In [ ]:
# df5 = pd.read_csv('../data/processed/df4_processed.csv', float_precision = 'round_trip')
df5 = df4.copy()
df5.shape

In [ ]:
df5.info()

In [ ]:
del df4
gc.collect()

### 5.1.1 Describing

In [ ]:
# df3.hist();
num_att = df5.select_dtypes(include= ['int','float'])
d1 = pd.DataFrame( num_att.min() ).T
d2 = pd.DataFrame( num_att.max() ).T
d3 = pd.DataFrame( num_att.max() - num_att.min() ).T
d4 = pd.DataFrame( num_att.mean() ).T
d5 = pd.DataFrame( num_att.median() ).T
d6 = pd.DataFrame( num_att.std() ).T
d7 = pd.DataFrame( num_att.skew() ).T
d8 = pd.DataFrame( num_att.kurtosis() ).T

desc = pd.concat( [d1, d2, d3, d4, d5, d6, d7, d8] ).T
desc.columns = ['min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis' ]
desc.style.format( "{:,.2f}" )

# 6.0 Feature Selection

## 6.3 Selected Embedding Space

In [ ]:
# selecting most important features. Removing ('avg_basket_size', 'avg_days_bw_purchases', 'avg_return_revenue', 'date_range)
cols_selected = ['qty_items','avg_ticket','num_returns','returns_revenue','avg_unique_prods','qty_returned','num_purchases','recency','frequency']

# defining independent and dependent variable
X = df5[cols_selected].copy()
y = df5['revenue']

# defining random forest regresor for embedding
rf = RandomForestRegressor(random_state = 42, min_samples_split = 20) 
rf.fit(X, y)

# get leafs for each observation
df_leaf = pd.DataFrame( rf.apply(X) )

# defining umap algorithm
n_neighbors = 50
umap = UMAP(n_neighbors = n_neighbors, random_state = 42, metric = 'manhattan')  
res_umap = umap.fit_transform(df_leaf)
metric = umap.metric

# storing embedding results on a dataframe
df_tree = df5.copy()
# df_tree = pd.DataFrame()
df_tree['embedding_x'] =  res_umap[:,0]
df_tree['embedding_y'] =  res_umap[:,1]

# ploting results
cmap = sns.color_palette('RdBu_r', as_cmap = True)
plt.title('UMAP - Neighbors = {} - Distance = {}'.format(n_neighbors, metric) )
sns.scatterplot( data = df_tree, x = 'embedding_x', y = 'embedding_y', hue = 'revenue', palette = cmap);

In [ ]:
X.dtypes

In [ ]:
X.head(10)

In [ ]:
df_leaf.head()

In [ ]:
df_leaf.to_csv('../data/df_tree.csv', index = False)

In [ ]:
res_umap[:10]

In [ ]:
res_umap[-10:]

# 7.0 Model Testing

In [ ]:
# defining two spaces of data (embedded and rescaled) to test models

# tree embedded data
df_tree = pd.DataFrame()
df_tree['embedding_x'] =  res_umap[:,0]
df_tree['embedding_y'] =  res_umap[:,1]

# rescaled data
df7 = df5.copy()

In [ ]:
del X, y, df5
gc.collect()

# 8.0 Model Training

In [ ]:
df7.shape

In [ ]:
# defining model 
db_model = DBSCAN(eps = 2.1, min_samples=50)

# applying model 
labels = db_model.fit_predict(df_tree)

df_result = df7.reset_index(drop = True).copy()

# concatenating results
df_result = pd.concat( [df_result, df_tree], axis = 1 )

# defining cluster
df_result['cluster'] = labels.astype(str)

# mapping cluster names
names = {"4" : "Insiders",
         "1" : "Potential Loyalists",
         "0" : "Loyal Customers",
         "2" : "Promissing 1",
         "8" : "Need Attention 1",
         "6" : "Promising 2",
         "7" : "Need Attention 2",
         "5" : "Hibernating 1",
         "10" : "Churn 1",
         "3" : "Price Sensitive",
         "9" : "Churn 2",
         "11" : "Hibernating 2",
         "-1" : 'Noise' }

df_result['cluster_name'] = df_result['cluster'].map(names)

df_result.rename( columns = {'date_range': 'tenure'}, inplace = True)
df_result.head()
df_result.shape

In [ ]:
sns.scatterplot(data = df_result, x = 'embedding_x', y = 'embedding_y', hue = 'cluster_name', palette = 'bright');

# 9.0 Cluster Analysis

## 9.4 Business Perspective

### 9.4.1 Clusters Table

In [ ]:
# number of customers per cluster
n_cust = df_result.copy()
n_cust = n_cust.groupby('cluster').count().reset_index().rename(columns = {'customer_id' : 'num_of_customers'})
n_cust['perc_of_customer'] = n_cust['num_of_customers']/n_cust['num_of_customers'].sum()*100
n_cust = n_cust[['cluster', 'num_of_customers', 'perc_of_customer']]

In [ ]:
# calculating mean values for each cluster attribute
df_res = df_result.copy()

# calculating mothly frequency. Did not like the results of this
# df_res['frequency'] = df_res['frequency']*30

# add cluster to df
result = df_res.groupby('cluster').mean().reset_index()
result

# add nº of customers
result = result.merge( n_cust, on = 'cluster', how = 'inner')
# Based on the RFM Analysis I gave names to clusters 
result['cluster_name'] = result['cluster'].map(names)
result_reduced = result[['cluster_name', 'num_of_customers', 'perc_of_customer', 'recency', 'revenue', 'avg_ticket', 'num_purchases', 'tenure', 'frequency', 'num_returns', 'avg_return_revenue' ]].copy()

#formatting output
result_reduced.sort_values('revenue', ascending = False, inplace = True )
result_reduced['perc_of_customer'] = result_reduced['perc_of_customer'].apply(lambda x: "{:.2f}%".format(x) )
formatter = {"recency" : "{:,.2f}",
"revenue" : "{:,.2f}",
"avg_ticket" : "{:,.2f}",
"num_purchases" : "{:,.2f}",
"tenure" : "{:,.2f}",
"frequency" : "{:,.4f}",
"num_returns" : "{:,.2f}",
"avg_return_revenue" : "{:,.2f}"
}
result_reduced.style.format( formatter = formatter )

In [ ]:
# # calculating mean values for each cluster attribute
# df_res = df_result.copy()

# # calculating mothly frequency. Did not like the results of this
# # df_res['frequency'] = df_res['frequency']*30

# # add cluster to df
# result = df_res.groupby('cluster').mean().reset_index()
# result

# # add nº of customers
# result = result.merge( n_cust, on = 'cluster', how = 'inner')
# # Based on the RFM Analysis I gave names to clusters 
# result['cluster_name'] = result['cluster'].map(names)
# result_reduced = result[['cluster_name', 'num_of_customers', 'perc_of_customer', 'recency', 'revenue', 'avg_ticket', 'num_purchases', 'tenure', 'frequency', 'num_returns', 'avg_return_revenue' ]].copy()

# #formatting output
# result_reduced.sort_values('revenue', ascending = False, inplace = True )
# result_reduced['perc_of_customer'] = result_reduced['perc_of_customer'].apply(lambda x: "{:.2f}%".format(x) )
# formatter = {"recency" : "{:,.2f}",
# "revenue" : "{:,.2f}",
# "avg_ticket" : "{:,.2f}",
# "num_purchases" : "{:,.2f}",
# "tenure" : "{:,.2f}",
# "frequency" : "{:,.4f}",
# "num_returns" : "{:,.2f}",
# "avg_return_revenue" : "{:,.2f}"
# }
# result_reduced.style.format( formatter = formatter )

# 11 Model Deployment

## 11.1. Inserting results on sqlite db

In [ ]:
#  connect to db, if doesn't exist create one
# con = sqlite3.connect('../src/features/insiders.db')

In [ ]:
# cur = con.cursor()

# cur.execute(
#     """
    # CREATE TABLE insiders (
    #     customer_id               INTEGER,
    #     country                   TEXT,
    #     recency                   INTEGER,
    #     avg_days_bw_purchases     REAL,
    #     num_purchases             INTEGER,
    #     tenure                    INTEGER,
    #     frequency                 REAL,
    #     revenue                   REAL,
    #     avg_ticket                REAL,
    #     qty_items                 INTEGER,
    #     avg_basket_size           REAL,
    #     avg_unique_prods          REAL,
    #     returns_revenue           REAL,
    #     avg_return_revenue        REAL,
    #     num_returns               REAL,
    #     qty_returned              REAL,
    #     embedding_x               REAL,
    #     embedding_y               REAL,
    #     cluster                   TEXT,
    #     cluster_name              TEXT
#     );
#     """
# )

# con.commit()

# # con.close()



In [ ]:
# df_result.dtypes

In [ ]:
# # con.execute(
# #     """
# #     SELECT name
# #     FROM sqlite_schema
# #     WHERE type ='table'
# #     """
# # )
# cur.execute(
# """SELECT 
#         name
#     FROM 
#         sqlite_master
#     WHERE 
#         type ='table' 
#         ;"""
#         )
# # con.commit()
# #  AND name NOT LIKE 'sqlite_%'
# cur.fetchone()

In [ ]:
# con.execute(
#     """
#     DROP TABLE insiders
#     """
# )
# con.commit()

# # con.close()


In [ ]:
# conn = create_engine('sqlite:////home/humberto/DS/insiders_clustering/src/features/insiders.db', echo=False)
# df_result.to_sql('insiders', con = conn, if_exists='append', index = False)

In [ ]:
# df_sql = pd.read_sql('insiders', con = conn)
# print(df_sql.iloc[0,8])
# print(df_result.iloc[0,8])
# df_sql.head()

## 11.2 Inserting results to mysql db on AWS

In [ ]:
# adding last update time to data that will be inserted on DB

now = datetime.now().strftime( "%Y-%m-%d %H:%M:%S" )

df_result['last_update'] = now

In [ ]:
df_result.head()

In [ ]:
# # reading credentials

# secrets_json = open('../secrets.json')
# secrets = json.load(secrets_json)

# dialect =   secrets["dialect"]
# driver =    secrets["driver"]
# host =      secrets["host"]
# username =  secrets["username"]
# password =  secrets["password"]
# port =      secrets["port"]
# database =  secrets["database"]

# url = "{}+{}://{}:{}@{}:{}/{}".format(dialect, driver, username, password, host, port, database)

# engine = create_engine(url = url, echo = True) #, pool_pre_ping = True

In [ ]:
# read_query = """
#     SELECT * FROM insiders i
# """
# # engine.execute
# df_sql = pd.read_sql_query(read_query, con = engine)

In [ ]:
# if we had to create table manually we could use this
# create_table = """
#     CREATE TABLE insiders (
#     customer_id               INTEGER,
#     country                   TEXT,
#     recency                   INTEGER,
#     avg_days_bw_purchases     REAL,
#     num_purchases             INTEGER,
#     tenure                    INTEGER,
#     frequency                 REAL,
#     revenue                   REAL,
#     avg_ticket                REAL,
#     qty_items                 INTEGER,
#     avg_basket_size           REAL,
#     avg_unique_prods          REAL,
#     returns_revenue           REAL,
#     avg_return_revenue        REAL,
#     num_returns               REAL,
#     qty_returned              REAL,
#     embedding_x               REAL,
#     embedding_y               REAL,
#     cluster                   TEXT,
#     cluster_name              TEXT
#     )
# """

In [ ]:
# inserting data
# df_result.to_sql('insiders', con = engine, if_exists='replace', index = False )

In [ ]:
# drop_table_query = """
#     DROP TABLE insiders
# """
# engine.execute(drop_table_query)


## 11.3 Inserting results to mysql db on heroku

In [ ]:
# adding last update time to data that will be inserted on DB

now = datetime.now().strftime( "%Y-%m-%d %H:%M:%S" )

df_result['last_update'] = now
df_result.head()

In [ ]:
# reading credentials

secrets_json = open('../secrets-heroku.json')
secrets = json.load(secrets_json)

dialect =   secrets["dialect"]
driver =    secrets["driver"]
host =      secrets["host"]
username =  secrets["username"]
password =  secrets["password"]
port =      secrets["port"]
database =  secrets["database"]

url = "{}+{}://{}:{}@{}:{}/{}".format(dialect, driver, username, password, host, port, database)

url 

dbschema = 'public'

# connecting to db
engine = create_engine(url = url, echo = True, connect_args={'options': '-csearch_path={}'.format(dbschema)} ) #, pool_pre_ping = True
conn = engine.connect()

In [ ]:
# import os
# os.environ['DATABASE_URL']

In [ ]:
df_result.to_sql('insiders_result', con = engine, if_exists='replace', index = False )

# if we have precision problems we can use:
# from sqlalchemy.dialects.postgresql import DOUBLE_PRECISION
# dtypes = {'avg_days_bw_purchases' : DOUBLE_PRECISION}
# df_result.to_sql('insiders_result', con = engine, if_exists='replace', index = False, dtype = dtypes )

In [ ]:
query = """
    SELECT *
    FROM insiders_result
"""

df_sql = pd.read_sql_query(query, con = engine)

In [ ]:
conn.invalidate()
engine.dispose()

In [ ]:
sns.scatterplot( data = df_sql, x = 'embedding_x', y = 'embedding_y', hue = 'cluster_name', palette = 'husl');

In [ ]:
sns.scatterplot( data = df_result, x = 'embedding_x', y = 'embedding_y', hue = 'cluster_name', palette = 'husl');

In [ ]:
df_sql.cluster_name.unique()

- verify aws packages OK
- automate reports with:
    - crontab 
    - github
- make a visualization dashboard
- automate ec2 to turn on and off